In [1]:
import numpy as np

In [10]:
# load the train data :

allImagesNP = np.load("/Users/sunilkumarpradhan/Personal_Projects/Github_Profile/Object_Segmetation_Project/Object_Segmentation_data/Unet-Animals-train-images.npy")
maskImagesNP = np.load("/Users/sunilkumarpradhan/Personal_Projects/Github_Profile/Object_Segmetation_Project/Object_Segmentation_data/Unet-Animals-train-mask.npy")

print(allImagesNP.shape)
print(maskImagesNP.shape)

Weight = 128
Width = 128
numOfCategories = 3


(3680, 128, 128, 3)
(3680, 128, 128)


In [12]:
# change the values of the mask from integer to categorical
from keras import utils
#from keras.utils import np_utils

In [14]:
# update to  categorical only for the first(!!!!) mask
# lets display the result before and after

test = maskImagesNP[0]
test = test -1 # convert the values from 1-3 to 0-2
test2 = utils.to_categorical(test, num_classes=numOfCategories) # 3

print(test)
print(test2)

[[1 1 1 ... 1 1 1]
 [1 1 1 ... 1 1 1]
 [1 1 1 ... 1 1 1]
 ...
 [1 1 1 ... 1 1 1]
 [1 1 1 ... 1 1 1]
 [1 1 1 ... 1 1 1]]
[[[0. 1. 0.]
  [0. 1. 0.]
  [0. 1. 0.]
  ...
  [0. 1. 0.]
  [0. 1. 0.]
  [0. 1. 0.]]

 [[0. 1. 0.]
  [0. 1. 0.]
  [0. 1. 0.]
  ...
  [0. 1. 0.]
  [0. 1. 0.]
  [0. 1. 0.]]

 [[0. 1. 0.]
  [0. 1. 0.]
  [0. 1. 0.]
  ...
  [0. 1. 0.]
  [0. 1. 0.]
  [0. 1. 0.]]

 ...

 [[0. 1. 0.]
  [0. 1. 0.]
  [0. 1. 0.]
  ...
  [0. 1. 0.]
  [0. 1. 0.]
  [0. 1. 0.]]

 [[0. 1. 0.]
  [0. 1. 0.]
  [0. 1. 0.]
  ...
  [0. 1. 0.]
  [0. 1. 0.]
  [0. 1. 0.]]

 [[0. 1. 0.]
  [0. 1. 0.]
  [0. 1. 0.]
  ...
  [0. 1. 0.]
  [0. 1. 0.]
  [0. 1. 0.]]]


In [16]:
# run the process for the all mask Numpy array
maskImagesNP = maskImagesNP - 1
maskForTheModel = utils.to_categorical(maskImagesNP , num_classes=numOfCategories)

print("print the type after the convert :")
print(maskForTheModel.dtype)
maskForTheModel = maskForTheModel.astype(int) # convert from float to integer
print(maskForTheModel.dtype)


print the type after the convert :
float64
int64


In [18]:
# split train and test
from sklearn.model_selection import train_test_split

X_train, X_val , y_train , y_val = train_test_split(allImagesNP, maskForTheModel, test_size=0.1 , random_state=42)
print("X_train , X_val , y_train , y_val --------->>>>  shapes :")

print(X_train.shape)
print(y_train.shape)

print(X_val.shape)
print(y_val.shape)

X_train , X_val , y_train , y_val --------->>>>  shapes :
(3312, 128, 128, 3)
(3312, 128, 128, 3)
(368, 128, 128, 3)
(368, 128, 128, 3)


In [20]:
# build the model

import tensorflow as tf
from Step02UnetModel import build_unet
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping

shape = (128,128,3)
num_classes = 3
lr = 1e-4 # 0.0001
batch_size = 4
epochs = 10


model = build_unet(shape , num_classes)
print(model.summary())

model.compile(loss="categorical_crossentropy", optimizer = tf.keras.optimizers.Adam(lr), metrics=['accuracy'])

stepsPerEpoch = np.ceil(len(X_train)/batch_size)
validationSteps = np.ceil(len(X_val)/batch_size)

best_model_file="e:/temp/Animals-Unet.h5"

callbacks = [
    ModelCheckpoint(best_model_file, verbose=1, save_best_only=True),
    ReduceLROnPlateau(monitor="val_loss", patience=3, factor=0.1, verbose=1, min_lr=1e-6),
    EarlyStopping(monitor='val_loss',patience=5 , verbose=1)
]

history = model.fit(X_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data = (X_val, y_val),
                    validation_steps = validationSteps,
                    steps_per_epoch = stepsPerEpoch,
                    shuffle=True,
                    callbacks=callbacks)

ModuleNotFoundError: No module named 'Step02UnetModel'